# Elastic Search
# 3. 데이터 모델링 (sql에서 스키마 만들기와 비슷함)
- 자세한 내용은 책을 참고 할 것


### 3.1.1 매핑인덱스 만들기
PUT movie_search1
{
  "settings": {
    "number_of_shards": 5,
    "number_of_replicas": 1
  },
  "mappings": {
    "_doc" : {
      "properties" : {
        "movieCd" : {
          "type" : "text",
          "analyzer" : "standard"
        },
        "movieNm" : {
          "type" : "text",
          "analyzer" : "standard"
        },
        "movieNmEn" : {
          "type" : "text",
          "analyzer" : "standard"
        },
        "prdtYear" : {
          "type" : "integer"
        },
        "openDt" : {
          "type" : "integer"
        },
        "typeNm" : {
          "type" : "keyword"
        },
        "prdtStatNm" : {
          "type" : "keyword"
        },
        "nationAlt" : {
          "type" : "keyword"
        },
        "genreAlt" : {
          "type" : "keyword"
        },
        "repNationNm" : {
          "type" : "keyword"
        },
        "repGenreNm" : {
          "type" : "keyword"
        },
        "companies" : {
          "properties" : {
            "companyCd" : {
              "type" : "keyword"
            },
            "companyNm" : {
              "type" : "keyword"
            }
          }
        },
        "directors" : {
          "properties" : {
            "peopleNm" : {
              "type" : "keyword"
            }
          }
        }
      }
    }
  }
}
- 결과 확인
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "movie_search1"
}



### 3.1.2 매핑확인
GET movie_search1/_mapping

- 매핑 정보 확인
{
  "movie_search1": {
    "mappings": {
      "_doc": {
        "properties": {
          "companies": {
            "properties": {
              "companyCd": {
                "type": "keyword"
              },
              "companyNm": {
                "type": "keyword"
              }
            }
          },
          "directors": {
            "properties": {
              "peopleNm": {
                "type": "keyword"
              }
            }
          },
          "genreAlt": {
            "type": "keyword"
          },
          "movieCd": {
            "type": "text",
            "analyzer": "standard"
          },
          "movieNm": {
            "type": "text",
            "analyzer": "standard"
          },
          "movieNmEn": {
            "type": "text",
            "analyzer": "standard"
          },
          "nationAlt": {
            "type": "keyword"
          },
          "openDt": {
            "type": "integer"
          },
          "prdtStatNm": {
            "type": "keyword"
          },
          "prdtYear": {
            "type": "integer"
          },
          "repGenreNm": {
            "type": "keyword"
          },
          "repNationNm": {
            "type": "keyword"
          },
          "typeNm": {
            "type": "keyword"
          }
        }
      }
    }
  }
}

## 3.2 메타 필드
### 3.2.1 _index : 해당 문서가 속한 인덱스의 이름을 담고 있음, 이를 이용해 인덱스명과 해당 인덱스에 몇 개의 문서가 있는지 확인 가능 *_search, _id, _uid, _source, _routing..... 가능
- 집계 API
POST movie_search/_search
{
  "size": 0,
  "aggs": {
    "indices": {
      "terms": {
        "field": "_index",
        "size": 10
      }
    }
  }
}
- 결과
{
  "took": 2,
  "timed_out": false,
  "_shards": {
    "total": 5,
    "successful": 5,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": 63069,
    "max_score": 0,
    "hits": []
  },
  "aggregations": {
    "indices": {
      "doc_count_error_upper_bound": 0,
      "sum_other_doc_count": 0,
      "buckets": [
        {
          "key": "movie_search",
          "doc_count": 63069
        }
      ]
    }
  }
}

POST movie_search/_search
{
  "query": {
    "term": {
      "movieCd": "20173732"
    }
  }
}

POST /_reindex
{
  "source": {
    "index": "movie_search",
    "query": {
      "match": {
        "movieCd": "20173732"
      }
    }
  },
  "dest": {
    "index": "reindex_movie"
  },
  "script": {
    "source" : "ctx._source.prdtYear++"
  }
}

## 3.3 필드 데이터 타입
### 3.3.1 Keyword 데이터 타입
- 정렬 필요, 집계 필요, 검색 시 필터링되는 항목에서 사용

### 3.3.2 Text 데이터 타입
- standard analyzer 사용함
- 정렬이나 집계가 필요하면 Text와 Keyword 타입 동시에 갖도록 멀티 필드로 설정

### 3.3.3 Array 데이터 타입
- 문자열 배열, 정수 배열, 객체 배열
- array안에 타입은 같아야 함

### 3.3.4 Numeric 데이터 타입
- 숫자 데이터 타입

### 3.3.5 Date 데이터 타입
- format 설정 가능

### 3.3.6 Range 데이터 타입

### 3.3.7 Boolean 데이터 타입
- true ("true") / false ("false")

### 3.3.8 Geo-Point 데이터 타입

### 3.3.9 IP 데이터 타입
- IPv4나 IPv6 모두 지정할 수 있음

### 3.3.10 Object 데이터 타입
- JSON 포맷의 문서는 내부 객체를 계층적으로 포함 가능함, 값으로 문서를 가지는 필드의 데이터를 Object 데이터 타입이라 함

### 3.3.11 Nested 데이터 타입
- 각 조건에 따른 쿼리문 작성 시에 array 데이터 타입 내부의 검색은 OR연산이 이루어지기 때문에 조건에 맞지 않아도 출력을 해준다. Nested 데이터 타입은 이를 위해 고안된 방법임

### 코드연습
- 텍스트와 키워드 타입 동시에 넣기
PUT movie_search_datatype/_mapping/_doc
{
  "properties" : {
    "movieComment" : {
      "type" : "text",
      "fields" : {
        "movieComment_keyword" : {
          "type" : "keyword"
        }
      }
    }
  }
}

- array 데이터 타입 넣기
PUT movie_search_datatype/_doc/1
{
  "title" : "해리포터와 마법사의 돌",
  "subtitleLang" : ["ko", "en"]
}

- object 데이터 타입
PUT movie_search_datatype/_mapping/_doc
{
  "properties" : {
    "companies" : {
      "properties" : {
        "companyName" : {
          "type" : "text"
        }
      }
    }
  }
}

PUT movie_search_datatype/_doc/_5
{
  "title" : "해리포터와 마법사의 돌",
  "companies" : {
    "companyName" : "위너브라더스"
  }
}

- Nested 데이터 타입
PUT movie_search_datatype/_mapping/_doc
{
  "properties" : {
    "companies_nested" : {
      "type" : "nested"
    }
  }
}

PUT movie_search_datatype/_doc/8
{
  "title" : "해리포터와 마법사의 돌",
  "companies_nested" : [
    {
    "companyCd" : "1",
    "companyName" : "위너브라더스"
    }, {
    "companyCd" : "2",
    "companyName" : "films"
    }
  ]
}
- 쿼리 실행 결과 (None)
POST movie_search_datatype/_search
{
  "query": {
    "nested": {
      "path": "companies_nested",
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "companies_nested.companyName": "위너브라더스"}
            }, {
              "match": {
                "companies_nested.companyCd": "2"}
            }
          ]
        }
      }
    }
  }
}

## 3.4 엘라스틱서치 분석기
### 3.4.1 텍스트 분석 개요
### 3.4.2 역색인 구조
### 3.4.3 분석기의 구조
- 인덱스 생성
PUT movie_analyzer
{
  "settings": {
    "index" : {
      "number_of_shards" : 5,
      "number_of_replicas" : 1
    },
    "analysis": {
      "analyzer": {
        "movie_lower_test_anlayzer" : {
          "type" : "custom",
          "tokenizer" : "standard", 
          "filter" : [
            "lowercase"
            ]
        },
        "movie_stop_test_analyzer" : {
          "type" : "custom",
          "tokenizer" : "standard", 
          "filter" : [
            "lowercase",
            "english_stop"
          ]
        }
      },
      "filter": {
        "english_stop" : {
          "type" : "stop",
          "stopwords" : "_english_"
        }
      }
    }
  },
  "mappings" : {
    "_doc" : {
      "properties" : {
        "title" : {
          "type" : "text",
          "analyzer" : "movie_stop_test_analyzer",
          "search_analyzer" : "movie_lower_test_anlayzer"
        }
      }
    }
  }
}

- 문서를 색인
PUT movie_analyzer/_doc/1
{
    "title" : "Harry Potter and the Chamber of Sercrets"
}
<br>결과 : [harry], [potter], [chamber], [secrets]

- 검색
POST movie_analyzer/_search
{
  "query" : {
    "query_string" : {
      "default_operator" : "AND",
      "query" : "Chamber of Secrets"
    }
  }
}
<br>결과 : [chamber], [of], [secrets]

### 3.4.4 전처리 필터
- Html strip char 필터
PUT movie_html_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "html_strip_analyzer" : {
          "tokenizer" : "keyword", 
          "char_filter" : [
            "html_strip_char_filter"
            ]
        }
      },
      "char_filter": {
        "html_strip_char_filter" : {
          "type" : "html_strip",
          "escaped_tags" : [
            "b"  
          ]
        }
      }
    }
  }
}
- 분석
POST movie_html_analyzer/_analyze
{
  "analyzer": "html_strip_analyzer",
  "text": "<span>Harry Potter</span> and the <b>Chamber</b> of Secrets"
}
- 결과 
"token": "Harry Potter and the <b>Chamber</b> of Secrets"

### 3.4.5 토크나이저 필터
- Standard 토크나이저
POST movie_analyzer/_analyze
{
  "tokenizer": "standard", 
  "text": "Harry Potter and the Chamber of Secrets"
}
- 결과 : [Harry, Potter, and, the, Chamber, of, Secrets]
<br><br>

- Whitespace 토크나이저
POST movie_analyzer/_analyze
{
  "tokenizer": "whitespace", 
  "text": "Harry Potter and the Chamber of Secrets"
}
- 결과 : [Harry, Potter, and, the, Chamber, of, Secrets]
<br><br>

- Ngram 토크나이저
PUT movie_ngram_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "ngram_analyzer" : {
          "tokenizer" : "ngram_tokenizer"
        }
      },
      "tokenizer": {
        "ngram_tokenizer" : {
          "type" : "ngram",
          "min_gram" : 3,
          "max_gram" : 3,
          "token_chars" : [
            "letter"  
          ]
        }
      }
    }
  }
}
- 실행
POST movie_ngram_analyzer/_analyze
{
  "tokenizer": "ngram_tokenizer", 
  "text": "Harry Potter and the Chamber of Secrets"
}
- 결과 : [Har, arr, rry, Pot, ott, ... Sec, erc, cre, ret, ets]
<br><br>

- Edge Ngram 토크나이저
PUT movie_engram_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "edge_ngram_analyzer" : {
          "tokenizer" : "edge_ngram_tokenizer"
        }
      },
      "tokenizer": {
        "edge_ngram_tokenizer" : {
          "type" : "edge_ngram",
          "min_gram" : 2,
          "max_gram" : 10,
          "token_chars" : [
            "letter"  
          ]
        }
      }
    }
  }
}
- 실행
POST movie_engram_analyzer/_analyze
{
  "tokenizer": "edge_ngram_tokenizer", 
  "text": "Harry Potter and the Chamber of Secrets"
}

- 결과 : [Ha, Har, Harr, Harry, Po, Pot, Pott, ... , Secret, Secrets]
<br><br>

### 3.4.6 토큰 필터
- Ascii Folding 토큰 필터 : 아스키 코드에 해당하지 않는 언어를 아스키 요소로 변경
- Lowercase 토큰 필터
PUT movie_lower_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "lower_analyzer" : {
          "tokenizer" : "standard",
          "filter" : [
              "lowercase"
          ]
        }
      }
    }
  }
}

- 실행
POST movie_lower_analyzer/_analyze
{
  "analyzer": "lower_analyzer", 
  "text": "Harry Potter and the Chamber of Secrets"
}

- 결과 : [harry, potter, and, the, chamber, of, secrets]
<br><br>

- Uppercase 토큰 필터
PUT movie_upper_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "uppercase_analyzer" : {
          "tokenizer" : "standard",
          "filter" : [
              "uppercase"
          ]
        }
      }
    }
  }
}
- 실행
POST movie_upper_analyzer/_analyze
{
  "analyzer": "uppercase_analyzer", 
  "text": "Harry Potter and the Chamber of Secrets"
}

- 결과 : [HARRY, POTTER, AND, THE, CHAMBER, OF, SECRETS]
<br><br>

- Stop 토큰 필터
PUT movie_stop_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "stop_filter_analyzer" : {
          "tokenizer" : "standard",
          "filter" : [
              "standard",
              "stop_filter"
          ]
        }
      },
      "filter" : {
        "stop_filter" : {
          "type" : "stop",
          "stopword" : [
            "and", "is", "the"
          ]  
        }
      }
    }
  }
}
- 실행
POST movie_stop_analyzer/_analyze
{
  "analyzer": "stop_filter_analyzer", 
  "text": "Harry Potter and the Chamber of Secrets"
}
- 결과 : [Harry, Potter, Chamber, of, Secrets]
<br><br>

- Stemmer 토큰 필터 : 영단어 원형으로 변환하는 필터
PUT movie_stem_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "stemmer_eng_analyzer" : {
          "tokenizer" : "standard",
          "filter" : [
              "standard",
              "lowercase",
              "stemmer_eng_filter"
          ]
        }
      },
      "filter" : {
        "stemmer_eng_filter" : {
          "type" : "stemmer",
          "name" : "english"
        }
      }
    }
  }
}
- 실행
POST movie_stem_analyzer/_analyze
{
  "analyzer": "stemmer_eng_analyzer", 
  "text": "Harry Potter and the Chamber of Secrets"
}

- 결과 : [harri, potter, and, the, chamber, of, secret]
<br><br>

- Synonym 토큰 필터
PUT movie_syno_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "synonym_analyzer" : {
          "tokenizer" : "whitespace",
          "filter" : [
              "synonym_filter"
          ]
        }
      },
      "filter" : {
        "synonym_filter" : {
          "type" : "synonym",
          "synonyms" : [
              "Harry => 해리"
          ]
        }
      }
    }
  }
}
- 실행
POST movie_syno_analyzer/_analyze
{
  "analyzer": "synonym_analyzer", 
  "text": "Harry Potter and the Chamber of Secrets"
}

- 결과 : [해리, Potter, and, the, Chamber, of, Secrets]
<br><br>

- Trim 토큰 필터
PUT movie_trim_analyzer
{
  "settings": {
    "analysis": {
      "analyzer": {
        "trim_analyzer" : {
          "tokenizer" : "keyword",
          "filter" : [
              "lowercase",
              "trim"
          ]
        }
      }
    }
  }
}

- 실행
POST movie_trim_analyzer/_analyze
{
  "analyzer": "trim_analyzer", 
  "text": "     Harry Potter and the Chamber of Secrets     "
}

- 결과 : [harry potter and the chamber of secrets]
<br><br>

### 3.4.7 동의어 사전
- 동의어 사전만들기 & 동의어 추가
- 동의어 치환하기
PUT movie_analyzer
{
  "settings": {
    "index": {
      "analysis": {
        "analyzer" : {
            "synonym_analyzer" : {
              "tokenizer" : "standard",
              "filter" : [
              "lowercase",
              "synonym_filter"
            ]
          }
        },
        "filter" : {
          "synonym_filter" : {
            "type" : "synonym",
            "ignore_case" : true,
            "synonyms_path" : "analysis/synonym.txt"
          }
        }
      }
    }
  }
}

- 실행
POST movie_analyzer/_analyze
{
  "analyzer": "synonym_analyzer", 
  "text": "Elasticsearch Harry Potter"
}

- 결과 : [elasticsearch, 엘라스틱서치, 해리, Potter]
- 참고 : 동의어 사전이 중간에 추가되는 경우, 아래와 같은 과정을 거친 후에 진행 (인덱스 Reload)
POST movie_analyzer/_close
POST movie_analyzer/_open

<br><br>

## 3.5 Document API 이해하기
- 책 참고